## Harmony Py Tutorial

This notebook shows a basic examples of a Harmony job, using a Harmony test Collection, to perform a combination of both spatial and temporal subsetting.

First, we import a few things that will help us create a request and display images. We then import the Harmony Py classes we need to make a request.

In [ ]:
import sys; sys.path.append('..')
!{sys.executable} -m pip install -q -r ../requirements/examples.txt

# Install harmony-py requirements.  Not necessary if you ran `pip install harmony-py` in your kernel  
!{sys.executable} -m pip install -q -r ../requirements/core.txt

import datetime as dt
from IPython.display import display, JSON
import rasterio
import rasterio.plot

from harmony import BBox, Client, Collection, Request, Environment

Now we create a Harmony Client object, letting it pick up our credentials from a `.netrc` file.

In [ ]:
harmony_client = Client(env=Environment.UAT)

Next, let's create a Collection object with the CMR collection id for the CMR collection we'd like to look at.

We then create a Request which specifies that collection, a `spatial` `BBox` describing the bounding box for the area we're interested in (we'll look at the ``BBox`` in other tutorials). In this case we're interested in looking at Alaska (and who wouldn't be?). We also include a date/time range to narrow down the data.

Because this data includes a lot of different variables, we limit it by passing in a list of `variable`s we're interested in; in this test collection we'll look at the blue variable. We include a `max_results` parameter to limit the results to the first 10 images just to get a sample of what things look like.

Next, we include a coordinate reference system (CRS) indicating we'd like to reproject the data into the [Arctic Polar Stereographic projection](https://epsg.io/3995). We also specify that we'd like the output to be in the GeoTIFF format with a resolution of 512x512 pixels.

Finally we check if the request we've created is valid.

In [ ]:
collection = Collection(id='C1234088182-EEDTEST')

request = Request(
    collection=collection,
    spatial=BBox(-165, 52, -140, 77),
    temporal={
        'start': dt.datetime(2010, 1, 1),
        'stop': dt.datetime(2020, 12, 30)
    },
    variables=['blue_var'],
    max_results=10,
    crs='EPSG:3995',
    format='image/tiff',
    height=512,
    width=512
    # concatenate=False (default is no concatenation)
)

request.is_valid()

Now that we have a request, we can submit it to Harmony using the Harmony Client object we created earlier. We'll get back an id for our request which we can use to find the job's status and get the results.

In [ ]:
job_id = harmony_client.submit(request)

Let's see how it's going. This will show the percentage complete in the `progress` field. (We use the JSON helper function to show the results in a nicer-to-look-at format).

In [ ]:
JSON(harmony_client.status(job_id))

Let's download the results to our system temp directory, overwriting files if they already exist. This returns us a list of `Future` objects. Each of these "stand in" for a file in our set of results. We can ask a `Future` for its result and when it's available, it will return the filename to us.

In [ ]:
results = harmony_client.download_all(job_id, directory='/tmp', overwrite=True)

Allright, now let's show some colorful Alaska images! Here we iterate over the results, asking each `Future` for its result, and then using `rasterio` to open the file and display the image.

In [ ]:
for r in results:
    rasterio.plot.show(rasterio.open(r.result()))